In [1]:
import markov_clustering as mc
import networkx as nx
import numpy as np
from  gensim import models
from scipy.sparse import *
from collections import defaultdict


try:
    from scipy.spatial import cKDTree as KDTree
except ImportError:
    _is_scipy_available = False
else:
    _is_scipy_available = True

expansion = 3  # 使得不同的区域之间的联系加强
inflation = 1.1  # 各个数据幂次方，强化紧密的点，弱化松散的点
loop_value = 1  # 自循环的概率
iterations = 1000
pruning_threshold = 0.005
pruning_frequency = 0.75
base_relevant=-1
def limit_frequency():
    #删除低于10个用户点击的商品code
    pw_Min10=[]
    with open('../data_rmcid3.txt', encoding='utf8') as file:
        for line in file:       
            pws=list(pw_idx for pw_idx in line.strip().split("\t"))
            pw=pws[0]
            num=int(pws[1])
            if num<10:
                pw_Min10.append(pw)
    code_Min10=[]
    with open('../id2pw_name.txt', encoding='utf8') as file:
        for line in file:       
            pws=list(pw_idx for pw_idx in line.strip().split("\t"))
            pw=pws[0]
            code=int(pws[1])
            if pw in pw_Min10:
                code_Min10.append(code)
    return code_Min10
def load_id1_name():
    id1_name={}
    for line in open('../cid1_name_code.csv', encoding='utf8'):
        cons = line.strip().split("\t")
        try:
            id1_name[cons[0]]=cons[1]
        except:
            print(cons[0])
    return id1_name
#载入pw序号及用户点击的次数
def rn_num():
    rn_num={}
    for line in open('../id1_pw_num', encoding='utf8'):
        cons = line.strip().split("\t")
        try:
            rn_num[int(cons[2])]=int(cons[3])
        except:
            print(cons[2])
    return rn_num
###对应对输入及MCL矩阵处理###
def load_id2pw(limit_code):
    id2pw={}
    #用来记录矩阵的行列数
    global all_line
    all_line=0
    i=1
    for line in open('../id2pw.txt', encoding='utf8'):
        all_line+=1
        cons = line.strip().split("\t")
        if i not in limit_code:
            try:
                a=vec_Glove[i]
                id2pw[i]=cons[0]
            except:
                i+=1
                continue
        i+=1
    return id2pw
def load_Glove_data():
    #输入glove生成的矩阵
    item={}
    with open('./vec_glv_475_rmcid3.txt', encoding='utf8') as file:
        for line in file:
            try:
                vec=[]
                pws=[pw_idx for pw_idx in line.strip().split(" ")]
                id=int(pws[0])
                for content in pws[1:]:
                    vec.append(float(content))
                item[id]=np.array(vec).T
            except:
                continue
    return item
def load_data_relevant(filename):
    edges={}
    with open(filename, encoding='utf8') as file:
        for line in file:
            item=line.split(":")
            key=int(item[0])
            cons = [pw for pw in item[1].strip().split(",") if pw!='']
            edges[key]=cons
    return edges
vec_Glove=load_Glove_data()
print("模型加载完毕")
code_Min10=limit_frequency()
print("限制的商品code加载完毕")
id2pw=load_id2pw(code_Min10)
id1_name=load_id1_name()
rn_num=rn_num()
print("cid1-pw对加载完毕")
#进行相关数据处理，减少输入数据量
edges=load_data_relevant('../base_relevant_rmcid3_'+str(base_relevant))
print("相关数据处理完毕，限制最大相关数") 

模型加载完毕
限制的商品code加载完毕
cid1-pw对加载完毕
相关数据处理完毕，限制最大相关数


In [ ]:
class MCL(object):
    def __init__(self, id2pw: dict):
        self.id2pw = id2pw
        self.cluster_name = "mcl_Glove_test"
        self.rbcids = []

    def run_from_mat(self, sim_mat):
        result = mc.run_mcl(sim_mat,
                            expansion=expansion,
                            inflation=inflation,
                            loop_value=loop_value,
                            iterations=iterations,
                            pruning_threshold=pruning_threshold,
                            pruning_frequency=pruning_frequency,
                            verbose=True
                            )  # run MCL with default parameters
        rbcids = mc.get_clusters(result)  # get clusters
        for item in rbcids:
            sub_item = []
            for i in range(len(item)):
                pw = self.id2pw.get(item[i])
                if pw is not None:
                    sub_item.append(pw)
            self.rbcids.append(sub_item)

    def save_cluster(self):
        file = open(self.cluster_name, 'w', encoding='utf8')
        file.write("#############\n")
        file.write("expansion={0}\ninflation={1}\n"
                   "loop_value={2}\niterations={3}\n"
                   "pruning_threshold={4}\n"
                   "pruning_frequency={5}\n"
                   "base_relevant={6}\n".format(expansion, inflation,
                                                    loop_value, iterations,
                                                    pruning_threshold,
                                                    pruning_frequency,
                                                    base_relevant))
        file.write("#############\n")
        for item in self.rbcids:
            for i in range(len(item)):
                pw = item[i]
                try:
                    cid1=pw.split('-')[0]
                    cid1_name=id1_name[cid1]
                    pw=cid1_name+'-'+pw.split('-')[1]
                    file.write(pw+", " if i < len(item)-1 else pw)
                except:
                    print(pw)
            file.write("\n")
        file.close()
def _func(edges):
    print(len(id2pw))
    sim_mat = lil_matrix(( all_line+1, all_line + 1))
    for i in id2pw.keys():
        lst = edges.get(i)
        if lst is not None:
            for j in lst:
                j=int(j)
                try:
                    vec1 = vec_Glove[i]
                    vec2 = vec_Glove[j]
                    val1 = np.sqrt(sum([item ** 2 for item in vec1]))
                    val2 = np.sqrt(sum([item ** 2 for item in vec2]))
                    sim = vec1.dot(vec2.T) / (val1 * val2)
                    if rn_num[i]>=100000 and rn_num[j]>=100000:
                        if sim > 0.4:
                            sim_mat[i, j] = sim
                            sim_mat[j, i] = sim
                    else:
                        if sim > 0.15:
                            sim_mat[i, j] = sim
                            sim_mat[j, i] = sim
                except Exception as e:
                    #print(e) 
                    continue
    return sim_mat

graph_dict=_func(edges)
#print(graph_dict.data)
print('相似矩阵计算完毕')
A=MCL(id2pw)
A.run_from_mat(graph_dict)
A.save_cluster()
print('All done')

301485
相似矩阵计算完毕
--------------------------------------------------
MCL Parameters
Expansion: 3
Inflation: 1.1
Pruning threshold: 0.005, frequency: 0.75 iteration
Convergence check: 1 iteration
Maximum iterations: 1000
Sparse matrix mode
--------------------------------------------------
Iteration 1
